In [1]:
import argparse
import glob
import os
import time

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

In [2]:
parser = {
    'test_path': '../data/test_hq/',
    'intermediate_path': '../intermediate/output/',
    'output_paths': ['output_dn_31',
                     'output_tn_31', 'output_tn_41',
                     'output_unet_27', 'output_unet_33', 'output_unet_38',
                     'output_unet_41'],
    'submission_name': 'subu11.csv',
    'log_every': 1000,
}
args = argparse.Namespace(**parser)

args.output_paths = [os.path.join(args.intermediate_path, x)
                     for x in args.output_paths]
args.img_names = sorted([x.split('/')[-1]
                         for x in glob.glob(args.test_path + '/*.jpg')])

In [3]:
def rle_encode(mask_image):
    pixels = mask_image.flatten()
    # We avoid issues with '1' at the start or end (at the corners of 
    # the original image) by setting those pixels to '0' explicitly.
    # We do not expect these to be non-zero for an accurate mask, 
    # so this should not harm the score.
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs

In [4]:
def get_mask(output_paths, index, weights=None):
    if weights is None:
        weights = [1.] * len(output_paths)
    
    img = 0
    for i, path in enumerate(output_paths):
        file = os.path.join(path, str(index)+'.jpg')
        image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        img = img + image * weights[i]
    return (img / sum(weights)) > 127

In [5]:
def get_vote_mask(output_paths, index, weights=None):
    if weights is None:
        weights = [1.] * len(output_paths)
    
    img = 0
    for i, path in enumerate(output_paths):
        file = os.path.join(path, str(index)+'.jpg')
        image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        img = img + (image > 127) * weights[i]
    return img > (sum(weights)/2)

In [6]:
if 1 == 1:
#    weights = [3., 3., 6., 2., 4., 6., 11.]
    weights = [3., 3., 6., 2., 4., 6., 11.]
    list_rle = []
    init_time = time.time()
    for i in range(1, len(args.img_names)+1):
        mask = get_mask(args.output_paths, i, weights)
        rle = rle_encode(mask)
        list_rle.append(' '.join(str(x) for x in rle))

        if i % args.log_every == 0:
            print("Done {:6} images in {:.0f}s"
                  .format(i, time.time()-init_time))
else:
    weights = [1., 0, 1., 0, 1., 1., 1.]
    list_rle = []
    init_time = time.time()
    #for i in range(1, len(args.img_names)+1):
    for i in range(1, 2):
        mask = get_vote_mask(args.output_paths, i, weights)
        rle = rle_encode(mask)
        list_rle.append(' '.join(str(x) for x in rle))

        if i % args.log_every == 0:
            print("Done {:6} images in {:.0f}s"
                  .format(i, time.time()-init_time))
            plt.imshow(mask)

Done   1000 images in 98s
Done   2000 images in 190s
Done   3000 images in 281s
Done   4000 images in 372s
Done   5000 images in 463s
Done   6000 images in 555s
Done   7000 images in 648s
Done   8000 images in 739s
Done   9000 images in 830s
Done  10000 images in 921s
Done  11000 images in 1012s
Done  12000 images in 1104s
Done  13000 images in 1196s
Done  14000 images in 1288s
Done  15000 images in 1380s
Done  16000 images in 1473s
Done  17000 images in 1562s
Done  18000 images in 1626s
Done  19000 images in 1690s
Done  20000 images in 1750s
Done  21000 images in 1810s
Done  22000 images in 1873s
Done  23000 images in 1959s
Done  24000 images in 2020s
Done  25000 images in 2079s
Done  26000 images in 2140s
Done  27000 images in 2199s
Done  28000 images in 2259s
Done  29000 images in 2320s
Done  30000 images in 2380s
Done  31000 images in 2440s
Done  32000 images in 2519s
Done  33000 images in 2612s
Done  34000 images in 2705s
Done  35000 images in 2797s
Done  36000 images in 2898s
Don

In [7]:
df = pd.DataFrame({"img": args.img_names, "rle_mask": list_rle})
submiss_path = os.path.join(args.intermediate_path, args.submission_name)
df.to_csv(submiss_path, index=False)